## Question 3
(35%) Neural Networks and Gaussian process. 

Predict the SP500 with the financial indicators assigned to your team in the google spreadsheet (ep, dp, de, dy, dfy, bm, svar,
ntis, infl, tbl , see RLab3 2 GWcausalSP500.R), some lagged series of these indicators and lags of the target using a Neural Network and a GP regression with your desired kernel.

Predict return, or price, or direction (up or down). For which target works best? Do some feature selection to disregard some variables, select appropriate lags: causality, (distance) correlation, VAR-test, Lasso ... (The script RLab5 GausProc.R can be of help. The dataset is goyal-welch2022Monthly.csv and work within the period 1927/2021.)

In [1]:
# bm,ep, ntis

In [2]:
# Packages
import os
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests

In [7]:
DATA_DIR = 'data'

In [26]:
file = 'goyal-welch2022Monthly.csv'
data = pd.read_csv(os.path.join(DATA_DIR, file)) #,
                   #usecols=['yyyymm', 'Index', 'b/m', 'ep', 'ntis'])

data['Index'] = data['Index'].apply(lambda x: x.replace(',', '')).astype(float)
data['date'] = pd.to_datetime(data['yyyymm'], format='%Y%m')

data = data[['date', 'D12', 'Rfree', 'b/m', 'ntis', 'Index']]

dateI = "1972-01-01"
dateF = "2021-12-31"
data = data.query(f'date >= "{dateI}" & date <= "{dateF}"')
data.head()

,date,D12,Rfree,b/m,ntis,Index
1212,1972-01-01,3.07,0.0029,0.635357,0.033075,103.94
1213,1972-02-01,3.07,0.0025,0.617586,0.033030,106.57
1214,1972-03-01,3.07,0.0027,0.645902,0.031723,107.20
1215,1972-04-01,3.07,0.0029,0.636784,0.032492,107.67
1216,1972-05-01,3.07,0.0030,0.632442,0.030657,109.53


In [30]:
# Compute log returns of SP500 (logret)
logret = np.diff(np.log(data['Index']))

# Compute IndexDiv
IndexDiv = data['Index'] + data['D12']

# Compute log returns of IndexDiv (logretdiv)
logretdiv = np.diff(np.log(IndexDiv))

# Compute log of Rfree
logRfree = np.log(data['Rfree'] + 1)[1:]

# Compute GSPCep
GSPCep = logretdiv - logRfree
GSPCep.name = 'GSPCep'

# Merge features with lags
lags = [1, 2, 3]
features = [GSPCep.shift(lag).rename(f'lag_{lag}') for lag in lags]
features.append((GSPCep ** 2).shift(1).rename('lag_1_sq'))
features.append((GSPCep ** 2).shift(2).rename('lag_2_sq'))

# Merge all features
Z = pd.concat([GSPCep] + features, axis=1)

print(Z)

        GSPCep     lag_1     lag_2     lag_3  lag_1_sq  lag_2_sq
1213  0.021783       NaN       NaN       NaN       NaN       NaN
1214  0.003033  0.021783       NaN       NaN  0.000475       NaN
1215  0.001357  0.003033  0.021783       NaN  0.000009  0.000475
1216  0.013661  0.001357  0.003033  0.021783  0.000002  0.000009
1217 -0.024350  0.013661  0.001357  0.003033  0.000187  0.000002
...        ...       ...       ...       ...       ...       ...
1807  0.028310  0.022302  0.021694  0.005415  0.000497  0.000471
1808 -0.047991  0.028310  0.022302  0.021694  0.000801  0.000497
1809  0.066062 -0.047991  0.028310  0.022302  0.002303  0.000801
1810 -0.008279  0.066062 -0.047991  0.028310  0.004364  0.002303
1811  0.042225 -0.008279  0.066062 -0.047991  0.000069  0.004364

[599 rows x 6 columns]
